In [1]:
import pandas as pd
X_train = pd.read_csv('X_train_norm_pca.csv')
X_test = pd.read_csv('X_test_norm_pca.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

X_train = pd.read_csv('X_train_norm_pca.csv')
X_test = pd.read_csv('X_test_norm_pca.csv')
y_train = pd.read_csv('y_train.csv').values.ravel()
y_test = pd.read_csv('y_test.csv').values.ravel()
base_models = [
    ('RandomForest', RandomForestClassifier(n_estimators=243, max_depth=27, min_samples_split=12,
                                            min_samples_leaf=1, max_features='log2', random_state=8)),

    ('CatBoost', CatBoostClassifier(iterations=480, depth=14, learning_rate=0.011574,
                                    l2_leaf_reg=3.8, border_count=86, verbose=0, random_seed=8)),

    ('ExtraTrees', ExtraTreesClassifier(n_estimators=153, max_depth=26, min_samples_split=4,
                                        min_samples_leaf=4, max_features=None, random_state=8)),

    ('SVM', SVC(kernel='rbf', C=0.12, gamma=0.0024, probability=True, random_state=8)),

    ('OvR_SVC', SVC(kernel='rbf', C=0.3051, gamma=0.0020, probability=True, 
                    random_state=8, decision_function_shape='ovr'))
]

meta_learner = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=8)

stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_learner, passthrough=True, cv=5)
stacking_model.fit(X_train, y_train)
y_pred = stacking_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Stacked Model Accuracy (RF Meta Learner): {accuracy:.4f}")
